In [ ]:
"""!pip install aeon
!pip install sktime
!pip install tsfresh
!pip install tslearn
!pip install PyWavelets"""

### To Do list


*   Comparar os resultados do 1NN contra o SVM+RF
*   Comparar os resultados dos classificadores Feature Based com o SVM+RF
*   Comparar os resultados do MetaClf_Conc contra o MetaClf_Dict



In [1]:
import pandas as pd
import numpy as np

from aeon.datasets import load_classification
from aeon.datasets.tsc_data_lists import univariate_equal_length
from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier, ShapeDTW, ElasticEnsemble

from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import MinimalFCParameters

from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import PiecewiseAggregateApproximation, SymbolicAggregateApproximation

import pywt

from sklearn.metrics import accuracy_score

from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB

from scipy.fftpack import fft
from numba import jit
from tqdm import tqdm
import timeit
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Transform data using TimeSeriesScalerMeanVariance and concatenate all transformed data
@jit
def transform_data(X):
    n_sax_symbols = int(X.shape[1] / 4)
    n_paa_segments = int(X.shape[1] / 4)

    X_fft = np.abs(fft(X, axis=1))

    coeffs_cA, coeffs_cD = pywt.dwt(X, 'db1', axis=1)
    X_dwt = np.hstack((coeffs_cA, coeffs_cD))

    paa = PiecewiseAggregateApproximation(n_segments=n_paa_segments)
    X_paa_ = paa.inverse_transform(paa.fit_transform(X))
    X_paa = X_paa_.reshape(X_paa_.shape[0], -1)

    sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
    X_sax_ = sax.inverse_transform(sax.fit_transform(X))
    X_sax = X_sax_.reshape(X_sax_.shape[0], -1)

    data = np.concatenate((X, X_fft, X_dwt, X_paa, X_sax), axis=1)
    data_concat = TimeSeriesScalerMeanVariance().fit_transform(data)
    data_concat.resize(data.shape[0], data.shape[1])

    return data_concat

In [5]:
@jit
def select_model(option, random_state):
    if option == '1nn':
        return KNeighborsTimeSeriesClassifier(distance='euclidean',
                                              n_neighbors=1,
                                              n_jobs=-1)
    elif option == '3nn':
        return KNeighborsTimeSeriesClassifier(distance='dtw',
                                              n_neighbors=3,
                                              n_jobs=-1)
    elif option == 'svm':
        return SVC(C = 100,
                   gamma=0.01,
                   kernel='linear',
                   probability=True)
    elif option == 'gbc':
        return GradientBoostingClassifier(n_estimators=5,
                                          random_state=random_state)
    elif option == 'nb':
        return GaussianNB()
    elif option == 'shape':
        return ShapeDTW(n_neighbors=1)
    elif option == 'ee':
        return ElasticEnsemble(n_jobs=-1,
                               random_state=random_state,
                               majority_vote=True)
    elif option == 'exrf':
        return ExtraTreesClassifier(n_estimators=250,
                                    criterion="entropy",
                                    class_weight="balanced",
                                    max_features="sqrt",
                                    n_jobs=-1,
                                    random_state=None)
    elif option == 'rd':
        return RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
    else:
        return RandomForestClassifier(n_estimators=200,
                                      criterion="entropy",
                                      class_weight="balanced",
                                      max_features="sqrt",
                                      n_jobs=-1,
                                      random_state=None)

In [19]:
@jit
def train_with_meta_classifier(X_train, y_train, base_option='svm', meta_option='random_forest', random_state=42):
    X_train_transformed = transform_data(X_train)

    loo = LeaveOneOut()
    loo.get_n_splits(X_train_transformed)

    # Treinar um modelo para todos os dados transformados
    model = select_model(base_option, random_state)
    for train_index, test_index in tqdm(loo.split(X_train_transformed), ascii=True, colour='red', desc="Training"):
        X_train_fold, X_test_fold = X_train_transformed[train_index], X_train_transformed[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
        model.fit(X_train_fold, y_train_fold)

    # Preparar dados para o meta-classificador
    meta_features = []
    for X_trans in X_train_transformed:
        instance_features = []
        proba = model.predict_proba(X_trans.reshape(1, -1)) # Reshape para compatibilidade com predict_proba
        proba /= np.sum(proba)
        instance_features.extend(proba.flatten())
        meta_features.append(instance_features)

    meta_features = np.array(meta_features)

    # Treinar o meta-classificador
    meta_classifier = select_model(meta_option, random_state)
    meta_classifier.fit(meta_features, y_train)

    return model, meta_classifier


In [20]:
@jit
def predict_with_meta_classifier(X_test, trained_base_model, trained_meta_classifier):
    predictions = []
    meta_features_test = []  # Inicialize uma lista para armazenar todos os meta-recursos dos dados de teste

    for i in tqdm(range(len(X_test)), ascii=True, colour='green', desc="Testing"):
        x_instance = X_test[i].reshape(1, -1)
        x_transformed = transform_data(x_instance)

        instance_features = []
        for X_trans in x_transformed:  # Iterar sobre as diferentes transformações
            proba = trained_base_model.predict_proba(X_trans.reshape(1, -1))
            proba /= np.sum(proba)
            instance_features.extend(proba.flatten())  # Estender a lista com todas as probabilidades

        meta_feature = np.array(instance_features).reshape(1, -1)
        predictions.append(trained_meta_classifier.predict(meta_feature)[0])  # Adicionar a previsão à lista de previsões

        meta_features_test.append(meta_feature.flatten())  # Adicionar meta-recursos da instância atual à lista

    # Converter a lista de meta-recursos dos dados de teste em um array numpy
    meta_features_test = np.array(meta_features_test)

    return predictions


In [21]:
"""
Caso o teste seja aplicado a todos os conjuntos de dados da UCR
univariate_list = list(univariate_equal_length)
univariate_list.sort()"""

In [22]:
dataset_quali_list = ['Adiac', 'Beef', 'Car', 'CBF', 'Coffee', 'DiatomSizeReduction', 'ECG200', 'ECGFiveDays', 'FaceFour','GunPoint','Lightning2', 'Lightning7', 'MedicalImages', 'MoteStrain', 'OliveOil', 'SonyAIBORobotSurface1','SonyAIBORobotSurface2', 'SyntheticControl', 'Trace', 'TwoPatterns']
dataset_full_list= ['Worms','FaceAll','SyntheticControl','SemgHandMovementCh2','Herring','GunPointAgeSpan','SmoothSubspace','SemgHandSubjectCh2','LargeKitchenAppliances','Plane','Fish','ScreenType','PhalangesOutlinesCorrect','CricketZ','MiddlePhalanxOutlineAgeGroup','ECG5000','Chinatown','ShapeletSim','MiddlePhalanxTW','Symbols','EOGHorizontalSignal','Ham','UMD','HouseTwenty','TwoPatterns','MiddlePhalanxOutlineCorrect','Wafer','Rock','DistalPhalanxTW','CricketY','SonyAIBORobotSurface1','FacesUCR','FiftyWords','Mallat','Strawberry','SwedishLeaf','ProximalPhalanxOutlineAgeGroup','DiatomSizeReduction','MixedShapesRegularTrain','Trace','ECGFiveDays','Lightning2','MoteStrain','SmallKitchenAppliances','GunPointOldVersusYoung','Wine','ECG200','ProximalPhalanxOutlineCorrect','WordSynonyms', 'RefrigerationDevices','Lightning7','Yoga','FaceFour','CinCECGTorso','Beef','OliveOil','ChlorineConcentration','ArrowHead','ToeSegmentation1','TwoLeadECG','ProximalPhalanxTW','InsectEPGSmallTrain','WormsTwoClass','PowerCons','Coffee','InsectEPGRegularTrain','GunPointMaleVersusFemale','DistalPhalanxOutlineCorrect','ItalyPowerDemand','InsectWingbeatSound','BME','NonInvasiveFetalECGThorax2','CricketX','Haptics','EOGVerticalSignal','MixedShapesSmallTrain','Meat','SemgHandGenderCh2','ToeSegmentation2','Adiac','Car','NonInvasiveFetalECGThorax1','FreezerSmallTrain','OSULeaf','GunPoint','Earthquakes','BirdChicken','HandOutlines','BeetleFly','SonyAIBORobotSurface2','CBF','ACSF1','DistalPhalanxOutlineAgeGroup','FreezerRegularTrain']
problematicos = ['Crop','EthanolLevel','ElectricDevices','FordB','ShapesAll','StarLightCurves','Phoneme', 'Computers','InlineSkate','PigAirwayPressure', 'PigCVP','FordA','MedicalImages','PigArtPressure', 'UWaveGestureLibraryX','UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'UWaveGestureLibraryAll']

In [28]:
from sklearn import preprocessing
accuracy_data = []

for dataset_name in dataset_quali_list:
    train, train_labels = load_classification(dataset_name, split='TRAIN')
    test, test_labels = load_classification(dataset_name, split='test')

    xtrain = train.reshape(train.shape[0], -1)
    xtest = test.reshape(test.shape[0], -1)
    #data_train = transform_data(xtrain)
    #data_test = transform_data(xtest)

    # Treino
    trained_base_models, meta_classifier = train_with_meta_classifier(xtrain, train_labels, base_option='1nn', random_state=42)
    # Teste
    predictions_test_meta = predict_with_meta_classifier(xtest, trained_base_models, meta_classifier)
    # Resultado
    test_accuracy_meta = np.mean(predictions_test_meta == test_labels)

    accuracy_data.append({'Dataset Name': dataset_name, 'Accuracy': test_accuracy_meta})

accuracy_df = pd.DataFrame(accuracy_data)


Training: 390it [00:00, 489.01it/s]
Testing: 100%|##########| 391/391 [00:23<00:00, 16.66it/s]
Training: 30it [00:00, 149.43it/s]
Testing: 100%|##########| 30/30 [00:01<00:00, 18.72it/s]
Training: 60it [00:00, 965.59it/s]
Testing: 100%|##########| 60/60 [00:03<00:00, 18.02it/s]
Training: 30it [00:00, 1865.19it/s]
Testing: 100%|##########| 900/900 [00:45<00:00, 19.72it/s]
Training: 28it [00:00, 1995.73it/s]
Testing: 100%|##########| 28/28 [00:01<00:00, 19.77it/s]
Training: 16it [00:00, ?it/s]
Testing: 100%|##########| 306/306 [00:16<00:00, 18.95it/s]
Training: 100it [00:00, 1488.57it/s]
Testing: 100%|##########| 100/100 [00:05<00:00, 19.82it/s]
Training: 23it [00:00, 1585.46it/s]
Testing: 100%|##########| 861/861 [00:43<00:00, 19.85it/s]
Training: 24it [00:00, 1618.77it/s]
Testing: 100%|##########| 88/88 [00:04<00:00, 18.96it/s]
Training: 50it [00:00, 2411.32it/s]
Testing: 100%|##########| 150/150 [00:07<00:00, 19.95it/s]
Training: 60it [00:00, 901.81it/s]
Testing: 100%|##########| 61/6

In [29]:
accuracy_df

,Dataset Name,Accuracy
0,Adiac,0.744246
1,Beef,0.700000
2,Car,0.766667
3,CBF,0.864444
4,Coffee,1.000000
5,DiatomSizeReduction,0.931373
6,ECG200,0.870000
7,ECGFiveDays,0.972125
8,FaceFour,0.784091
9,GunPoint,0.980000


In [ ]:
accuracy_df.to_csv('model_votingCLF+NN_CD.csv', index=False)